# Brusselator model

The dynamics of the oscillating reaction discovered by Belousov and Zhabotinsky, 
can be modeled through the so-called Brusselator model depending on two parameters:

$$
\left\{\begin{aligned}
d_t y_1 & = 1 - (b+1) y_1 + a y_1^2y_2\\
d_t y_2 & = b y_1 - a y_1^2y_2
\end{aligned}\right.
$$

In [5]:
import numpy as np

from scipy.integrate import solve_ivp

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import column
from bokeh.models import PrintfTickFormatter

from mylib.model import brusselator_model
import mylib.integration as integration

import time

output_notebook(hide_banner=True)

## Radau5 integration (python implementation)

In [6]:
def plot_radau5_sol():
    
    bm = brusselator_model(a=1, b=3)
    fcn = bm.fcn
    jac = bm.jac
    
    tini = 0. 
    tend = 20.
    
    yini = (1.5, 3)
    
    tol = 1.e-6
    t0 = time.time()
    sol_radau = solve_ivp(fcn, (tini, tend), yini, method="Radau", rtol=tol, atol=tol, jac=jac)
    t1 = time.time()
    
    sol_exa = solve_ivp(fcn, (tini, tend), yini, method="Radau", rtol=1.e-12, atol=1.e-12, t_eval=sol_radau.t, jac=jac)
    y1_err = np.abs(sol_exa.y[0] - sol_radau.y[0])
    y2_err = np.abs(sol_exa.y[1] - sol_radau.y[1])

    fig_sol = figure(x_range=(tini, tend), width=950, height=300, title="Solution")
    fig_sol.x(sol_radau.t, sol_radau.y[0], legend="y1", line_width=2)    
    fig_sol.x(sol_radau.t, sol_radau.y[1], legend="y2", line_width=2, color="Green")
    fig_sol.legend.location = "top_left"
    
    fig_err = figure(x_range=(tini, tend), y_axis_type="log", plot_height=300, plot_width=950, title="Global error")
    fig_err.yaxis[0].formatter = PrintfTickFormatter(format="%8.1e")
    fig_err.x(sol_radau.t[1:], y1_err[1:], line_width=2, legend="y1")
    fig_err.x(sol_radau.t[1:], y2_err[1:], line_width=2, color="Green", legend="y2")
    fig_err.legend.location = "top_left"
    
    show(column(fig_sol, fig_err))
    
    print(f"Time to integrate : {(t1-t0):.5f} s")
    print(f"Number of accepted steps : {(sol_radau.t.size-1):d}")
    print(f"Number of function evaluations : {sol_radau.nfev:d}")
    print(f"Number of evaluations of the jacobian : {sol_radau.njev:d}")
    print(f"Number of LU decompositions {sol_radau.nlu:d}")
    
plot_radau5_sol()

Time to integrate : 0.16505 s
Number of accepted steps : 278
Number of function evaluations : 2206
Number of evaluations of the jacobian : 51
Number of LU decompositions 228


## Runge-Kutta "3/8 method"

In [7]:
def plot_rk38_sol():
    
    bm = brusselator_model(a=1, b=3)
    fcn = bm.fcn  
    
    tini = 0. 
    tend = 20.
    
    yini = (1.5, 3)
    
    nt = 1751
    t0 = time.time()
    sol_rk38 = integration.rk38(tini, tend, nt, yini, fcn)
    t1 = time.time()
    
    sol_exa = solve_ivp(fcn, (tini, tend), yini, method="Radau", rtol=1.e-12, atol=1.e-12, t_eval=sol_rk38.t)
    y1_err = np.abs(sol_exa.y[0] - sol_rk38.y[0])
    y2_err = np.abs(sol_exa.y[1] - sol_rk38.y[1])

    fig_sol = figure(x_range=(tini, tend), width=950, height=300, title="Solution")
    fig_sol.x(sol_rk38.t, sol_rk38.y[0], legend="y1", line_width=2)    
    fig_sol.x(sol_rk38.t, sol_rk38.y[1], legend="y2", line_width=2, color="Green")
    fig_sol.legend.location = "top_left"
    
    fig_err = figure(x_range=(tini, tend), y_axis_type="log", plot_height=300, plot_width=950, title="Global error")
    fig_err.yaxis[0].formatter = PrintfTickFormatter(format="%8.1e")
    fig_err.x(sol_rk38.t[1:], y1_err[1:], line_width=2, legend="y1")
    fig_err.x(sol_rk38.t[1:], y2_err[1:], line_width=2, color="Green", legend="y2")
    fig_sol.legend.location = "top_left"

    
    show(column(fig_sol, fig_err))

    print(f"Time to integrate : {(t1-t0):.5f} s")
    print(f"Number of time steps : {(nt-1):d}")
    print(f"Number of function evaluations : {(4*(nt-1)):d}")

plot_rk38_sol()

Time to integrate : 0.08020 s
Number of time steps : 1750
Number of function evaluations : 7000


## Embedded method

In [8]:
def plot_rk_emb_sol():
    
    bm = brusselator_model(a=1, b=3)
    fcn = bm.fcn  
    
    tini = 0. 
    tend = 20.
    
    yini = (1.5, 3)
    
    tol = 2.5e-8
    t0 = time.time()
    sol_rk_emb = integration.rk_embedded(tini, tend, yini, fcn, tol)
    t1 = time.time()

    sol_exa = solve_ivp(fcn, (tini, tend), yini, method="Radau", rtol=1.e-12, atol=1.e-12, t_eval=sol_rk_emb.t)
    y1_err = np.abs(sol_exa.y[0] - sol_rk_emb.y[0])
    y2_err = np.abs(sol_exa.y[1] - sol_rk_emb.y[1])

    fig_sol = figure(x_range=(tini, tend), width=950, height=300, title="Solution")
    fig_sol.x(sol_rk_emb.t, sol_rk_emb.y[0], legend="y1", line_width=2)    
    fig_sol.x(sol_rk_emb.t, sol_rk_emb.y[1], legend="y2", line_width=2, color="Green")
    fig_sol.legend.location = "top_left"
    
    fig_err = figure(x_range=(tini, tend), y_axis_type="log", plot_height=300, plot_width=950, title="Global error")
    fig_err.yaxis[0].formatter = PrintfTickFormatter(format="%8.1e")
    fig_err.x(sol_rk_emb.t[1:], y1_err[1:], line_width=2, legend="y1")
    fig_err.x(sol_rk_emb.t[1:], y2_err[1:], line_width=2, color="Green", legend="y2")
    fig_sol.legend.location = "top_left"
    
    show(column(fig_sol, fig_err))

    print(f"Time to integrate : {(t1-t0):.5f} s")
    print(f"Number of accepted steps : {(sol_rk_emb.t.size-1):d}")
    print(f"Number of function evaluations : {sol_rk_emb.nfev:d}")

plot_rk_emb_sol()

Time to integrate : 0.06300 s
Number of accepted steps : 676
Number of function evaluations : 2769


## Dopri5 method

In [9]:
def plot_dopri5_sol():
    
    bm = brusselator_model(a=1, b=3)
    fcn = bm.fcn
    jac = bm.jac
    
    tini = 0. 
    tend = 20.
    
    yini = (1.5, 3)
    
    tol = 5.e-9
    t0 = time.time()
    sol_dopri5 = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=tol, atol=tol)
    t1 = time.time()
    
    sol_exa = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-12, atol=1.e-12, t_eval=sol_dopri5.t)
    y1_err = np.abs(sol_exa.y[0] - sol_dopri5.y[0])
    y2_err = np.abs(sol_exa.y[1] - sol_dopri5.y[1])

    fig_sol = figure(x_range=(tini, tend), width=950, height=300, title="Solution")
    fig_sol.x(sol_dopri5.t, sol_dopri5.y[0], legend="y1", line_width=2)    
    fig_sol.x(sol_dopri5.t, sol_dopri5.y[1], legend="y2", line_width=2, color="Green")
    fig_sol.legend.location = "top_left"
    
    fig_err = figure(x_range=(tini, tend), y_axis_type="log", plot_height=300, plot_width=950, title="Global error")
    fig_err.yaxis[0].formatter = PrintfTickFormatter(format="%8.1e")
    fig_err.x(sol_dopri5.t[1:], y1_err[1:], line_width=2, legend="y1")
    fig_err.x(sol_dopri5.t[1:], y2_err[1:], line_width=2, color="Green", legend="y2")
    fig_sol.legend.location = "top_left"

    show(column(fig_sol, fig_err))
    
    print(f"Time to integrate : {(t1-t0):.5f} s")
    print(f"Number of accepted steps : {(sol_dopri5.t.size-1):d}")
    print(f"Number of function evaluations : {sol_dopri5.nfev:d}")
    
plot_dopri5_sol()

Time to integrate : 0.03946 s
Number of accepted steps : 300
Number of function evaluations : 2102


## Radau method (fortran implementation)

In [10]:
def plot_radau5_fortran_sol():
    
    bm = brusselator_model(a=1, b=3)
    fcn = bm.fcn
    fcn_radau = bm.fcn_radau
    jac = bm.jac
    
    tini = 0. 
    tend = 20.
    
    yini = (1.5, 3)
    
    tol = 5.e-8
    t0 = time.time()
    sol_radau = integration.radau5(tini, tend, yini, fcn_radau, njac=2, rtol=tol, atol=tol, iout=1)
    t1 = time.time()
    
    sol_exa = solve_ivp(fcn, (tini, tend), yini, method="Radau", rtol=1.e-12, atol=1.e-12, t_eval=sol_radau.t, jac=jac)
    y1_err = np.abs(sol_exa.y[0] - sol_radau.y[0])
    y2_err = np.abs(sol_exa.y[1] - sol_radau.y[1])

    fig_sol = figure(x_range=(tini, tend), width=950, height=300, title="Solution")
    fig_sol.x(sol_radau.t, sol_radau.y[0], legend="y1", line_width=2)    
    fig_sol.x(sol_radau.t, sol_radau.y[1], legend="y2", line_width=2, color="Green")
    fig_sol.legend.location = "top_left"
    
    fig_err = figure(x_range=(tini, tend), y_axis_type="log", plot_height=300, plot_width=950, title="Global error")
    fig_err.yaxis[0].formatter = PrintfTickFormatter(format="%8.1e")
    fig_err.x(sol_radau.t[1:], y1_err[1:], line_width=2, legend="y1")
    fig_err.x(sol_radau.t[1:], y2_err[1:], line_width=2, color="Green", legend="y2")
    fig_err.legend.location = "top_left"
    
    show(column(fig_sol, fig_err))
    
    print(f"Time to integrate : {(t1-t0):.5f} s")
    print(f"Number of function evaluations : {sol_radau.nfev:d}")
    print(f"Number of jacobian evaluations : {sol_radau.njev:d}")
    print(f"Number of computed steps : {sol_radau.nstep:d}")
    print(f"Number of accepted steps : {sol_radau.naccpt:d}")
    print(f"Number of rejected steps : {sol_radau.nrejct:d}")
    print(f"Number of LU decompositions : {sol_radau.ndec:d}")
    print(f"Number of forward-backward substitutions : {sol_radau.nsol:d}")    
    
plot_radau5_fortran_sol()

Time to integrate : 0.00583 s
Number of function evaluations : 1821
Number of jacobian evaluations : 214
Number of computed steps : 258
Number of accepted steps : 249
Number of rejected steps : 9
Number of LU decompositions : 239
Number of forward-backward substitutions : 524
